In [ ]:
import pandas as pd

In [ ]:
circuits = pd.read_csv('circuits.csv', encoding='ISO-8859-1')
constructor_results = pd.read_csv('constructorResults.csv', encoding='ISO-8859-1' )
constructor_standings = pd.read_csv('constructorStandings.csv', encoding='ISO-8859-1')
constructors = pd.read_csv('constructors.csv', encoding='ISO-8859-1')
driver_standings = pd.read_csv('driverStandings.csv', encoding='ISO-8859-1')
drivers = pd.read_csv('drivers.csv', encoding='ISO-8859-1')
lap_times = pd.read_csv('lapTimes.csv', encoding='ISO-8859-1')
pit_stops = pd.read_csv('pitStops.csv', encoding='ISO-8859-1')
qualifying = pd.read_csv('qualifying.csv', encoding='ISO-8859-1')
races = pd.read_csv('races.csv', encoding='ISO-8859-1')
results = pd.read_csv('results.csv', encoding='ISO-8859-1')
seasons = pd.read_csv('seasons.csv', encoding='ISO-8859-1')
status = pd.read_csv('status.csv', encoding='ISO-8859-1')

In [ ]:
df = pd.merge(results, races[['raceId', 'year', 'name', 'circuitId', 'date']], on='raceId', how='left')
df = pd.merge(df, circuits[['circuitId', 'location', 'country', 'lat', 'lng', 'alt']], on='circuitId', how='left')
df = pd.merge(df, drivers[['driverId', 'forename', 'surname', 'dob', 'nationality']], on='driverId', how='left')
df = pd.merge(df, constructors[['constructorId', 'name', 'nationality']], on='constructorId', how='left')


In [ ]:
df.rename(columns={
    'name_x': 'race',
    'country': 'country_of_race',
    'nationality_x': 'nationality_of_circuit',
    'forename': 'driver_name',
    'nationality_y': 'driver_nationality',
    'name_y': 'constructor',
    'nationality': 'constructor_nationality',
}, inplace=True)

In [ ]:
df['home_race'] = df['driver_nationality'] == df['country_of_race']

In [ ]:
df['F2_champion'] = None
df['Former_F1_World_Champion'] = None

In [ ]:
# Get previous race information
df['previous_race'] = df.groupby('driverId')['raceId'].shift(1)
df['points_in_previous_race'] = df.groupby('driverId')['points'].shift(1)
df['laps_completed_in_previous_races'] = df.groupby('driverId')['laps'].shift(1)
df['fastest_lap_from_last_race'] = df.groupby('driverId')['fastestLap'].shift(1)
df['status_finished_last_race'] = df.groupby('driverId')['statusId'].shift(1)

In [ ]:
pit_stop_avg = pit_stops.groupby(['raceId', 'driverId']).agg({'milliseconds': 'mean'}).reset_index()
pit_stop_avg.rename(columns={'milliseconds': 'average_time_lost_in_pits'}, inplace=True)
df = pd.merge(df, pit_stop_avg[['raceId', 'driverId', 'average_time_lost_in_pits']], on=['raceId', 'driverId'], how='left')



In [ ]:
df.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'year', 'race', 'circuitId', 'date',
       'location', 'country_of_race', 'lat', 'lng', 'alt', 'driver_name',
       'surname', 'dob', 'nationality_of_circuit', 'constructor',
       'driver_nationality', 'home_race', 'F2_champion',
       'Former_F1_World_Champion', 'previous_race', 'points_in_previous_race',
       'laps_completed_in_previous_races', 'fastest_lap_from_last_race',
       'status_finished_last_race', 'average_time_lost_in_pits'],
      dtype='object')

In [ ]:
df_final = df[[
    'resultId', 'raceId', 'year', 'race', 'country_of_race', 'nationality_of_circuit', 'driverId', 'number', 'driver_name',
    'F2_champion', 'Former_F1_World_Champion', 'driver_nationality', 'home_race', 'constructor', 'grid', 'positionOrder',
    'points', 'points_in_previous_race', 'laps', 'laps_completed_in_previous_races', 'time', 'milliseconds', 'fastestLap',
    'fastest_lap_from_last_race', 'average_time_lost_in_pits', 'statusId', 'status_finished_last_race'
]]

In [ ]:
# Export to CSV
df_final.to_csv('final_dataset.csv', index=False)